In [1]:
import tensorflow as tf

In [2]:
a = tf.constant(5, name="input_a")
b = tf.constant(3, name="input_b")
c = tf.mul(a, b, name="mul_c")
d = tf.add(a, b, name="add_d")
e = tf.add(c, d, name="add_e")

In [4]:
sess = tf.Session()
output = sess.run(e)

In [5]:
# TensorBoard용
writer = tf.train.SummaryWriter('./my_graph', sess.graph)

In [6]:
writer.close()
sess.close()

* input을 vector tensor로 만들기

In [7]:
a = tf.constant([5, 3], name="input_a")
b = tf.reduce_prod(a, name="prod_b")
c = tf.reduce_sum(a, name="sum_c")
d = tf.add(b, c, name="add_d")

In [9]:
sess = tf.Session()
sess.run(d)

23

In [10]:
sess.close()

* graph 만들기

In [11]:
g = tf.Graph()

In [ ]:
with g.as_default():
    a = tf.mul(2, 3)
    ...
    
# with 바깥에서는 다시 tensorflow가 만든 default_graph를 사용한다.

* feed_dict 예시

In [12]:
a = tf.add(2, 5)
b = tf.mul(a, 3)

sess = tf.Session()
replace_dict = {a: 15}

sess.run(b, feed_dict=replace_dict)

45

In [14]:
sess.close()

* session을 context manager로 사용하기.
* with 로 접근하면 session을 default graph와 비슷하게 사용할 수 있음.
* 이러면 Operation.run() or Tensor.eval()을 써야 하는데, 이는 sess.run()에 넘겨주는 것과 동일한 의미이다.

In [16]:
a = tf.constant(5)

sess = tf.Session()

with sess.as_default():
    print a.eval()
    
sess.close()

5


* placeholder 사용하기

In [17]:
import numpy as np

In [18]:
a = tf.placeholder(tf.int32, shape=[2], name="my_input")

b = tf.reduce_prod(a, name="prod_b")
c = tf.reduce_sum(a, name="sum_c")

d = tf.add(b, c, name="add_d")

In [19]:
sess = tf.Session()
input_dict = {a: np.array([5, 3], dtype=np.int32)}

sess.run(d, feed_dict=input_dict)

23

* variable 사용하기

In [20]:
my_var = tf.Variable(3, name="my_variable")

add = tf.add(5, my_var)
mul = tf.mul(8, my_var)

In [23]:
zeros = tf.zeros([2, 2])

ones = tf.ones([6])

uniform = tf.random_uniform([3, 3, 3], minval=0, maxval=10)

normal = tf.random_normal([3, 3, 3], mean=0.0, stddev=2.0)

In [24]:
# 변수 초기화하기
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

In [25]:
# 변수 값 변경하기
my_var = tf.Variable(1)

my_val_times_two = my_var.assign(my_var*2)

init = tf.initialize_all_variables()

sess = tf.Session()
sess.run(init)

print sess.run(my_val_times_two)

print sess.run(my_val_times_two)

print sess.run(my_val_times_two)

2
4
8


In [26]:
print sess.run(my_var.assign_add(5))

print sess.run(my_var.assign_sub(5))

13
8


* name scope 사용하기

In [6]:
with tf.name_scope("Scope_A"):
    a = tf.add(1, 2, name="A_add")
    b = tf.mul(a, 3, name="A_mul")
    
with tf.name_scope("Scope_B"):
    c = tf.add(4, 5, name="B_add")
    d = tf.mul(c, 6, name="B_mul")
    
e = tf.add(b, d, name="output")

In [8]:
writer = tf.train.SummaryWriter('./name_scope_1', graph=tf.get_default_graph())
writer.close()

* name scope 이중 적용하기

In [9]:
graph = tf.Graph()

with graph.as_default():
    in_1 = tf.placeholder(tf.float32, shape=[], name="input_a")
    in_2 = tf.placeholder(tf.float32, shape=[], name="input_b")
    const = tf.constant(3, dtype=tf.float32, name="static_value")
    
    with tf.name_scope("Transformation"):
        with tf.name_scope("A"):
            A_mul = tf.mul(in_1, const)
            A_out = tf.sub(A_mul, in_1)
            
        with tf.name_scope("B"):
            B_mul = tf.mul(in_2, const)
            B_out = tf.sub(B_mul, in_2)
            
        with tf.name_scope("C"):
            C_div = tf.div(A_out, B_out)
            C_out = tf.add(C_div, const)
            
        with tf.name_scope("D"):
            D_div = tf.div(B_out, A_out)
            D_out = tf.add(D_div, const)
            
    out = tf.maximum(C_out, D_out)
    
writer = tf.train.SummaryWriter('./name_scope_2', graph=graph)
writer.close()

* sum up

In [12]:
graph = tf.Graph()

with graph.as_default():
    
    with tf.name_scope("variables"):
        # variable to keep track of how many times the graph has been run
        global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')
        
        # variable that keeps track of the sum of all output values over time
        total_output = tf.Variable(0.0, dtype=tf.float32, trainable=False, name='total_output')
        
    with tf.name_scope("transformation"):
        
        with tf.name_scope("input"):
            a = tf.placeholder(tf.float32, shape=[None], name="input_placeholder_a")
            
        with tf.name_scope("intermediatr_layer"):
            b = tf.reduce_prod(a, name="product_b")
            c = tf.reduce_sum(a, name="sum_c")
            
        with tf.name_scope("output"):
            output = tf.add(b, c, name="output")
            
    with tf.name_scope("update"):
        update_total = total_output.assign_add(output)
        increment_step = global_step.assign_add(1)
        
    with tf.name_scope("summaries"):
        avg = tf.div(update_total, tf.cast(increment_step, tf.float32), name="average")
        
        tf.scalar_summary(b'Output', output, name="output_summary")
        tf.scalar_summary(b'Sum of outputs over time', update_total, name='total_summary')
        tf.scalar_summary(b'Average of outputs over time', avg, name='average_summary')
        
    with tf.name_scope('global_ops'):
        init = tf.initialize_all_variables()
        merged_summaries = tf.merge_all_summaries()

In [13]:
sess = tf.Session(graph=graph)
writer = tf.train.SummaryWriter('./improved_graph', graph)

In [14]:
sess.run(init)

In [15]:
def run_graph(input_tensor):
    feed_dict = {a: input_tensor}
    _, step, summary = sess.run([output, increment_step, merged_summaries], feed_dict=feed_dict)
    writer.add_summary(summary, global_step=step)

In [16]:
run_graph([2, 8])
run_graph([3, 1, 3, 3])
run_graph([8])
run_graph([1, 2, 3])
run_graph([11, 4])
run_graph([4, 1])
run_graph([7, 3, 1])
run_graph([6, 3])
run_graph([0, 2])
run_graph([4, 5, 6])

In [17]:
writer.flush()

In [18]:
writer.close()
sess.close()